<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Opimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# SGD(Stochastic Gradient Decent)

확률적 경사하강법: 전체 데이터셋을 한번에 학습 시키는 것은 많은 컴퓨팅 자원을 필요로 하는 일 이기 때문에 데이터셋을 분할해서 학습을 진행하는 것을 확률적 경사하강법이라고 한다. 하지만 오차함수의 상수인 데이터가 계속해서 변하게 되므로 하강궤적이 불안정하게 된다.

분할한 데이터를 학습해서 전체 데이터셋을 모두 학습 완료하는것을 에폭이라고 한다.

In [ ]:
class SGD:

    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

# Momentum

물리학의 관선에서 아이디어를 얻어 적용한 기법으로 Gradient Decent에서 방향과 학습률을 적용했다면 Momentum에서는 관성을 추가하게 된다. Gradient Decent는 초기값에 따라 Local minimum이나 plateau에서 학습이 진행돼지 않는 문제가 있는 반해 Momentum은 관성을 이용해 빠져나올 수 있게 된다.
또 매개변수간 변화율으 차이가 크게 되는 경우 Gradient Decent의 궤적이 진동하며 매우 비효율적인 동선을 보이는데 Momentum을 사용하게 되면 이런 문제점도 약간 개선된다.

점화식은

$V_{n} = \alpha V_{n-1} - \eta \bigtriangledown f(x_n) $

$V_{-1} = 0 $

$X_{n+1} = X_n + V_n$

물리계에 비유하면 $\alpha V_{n-1}$가 관성이 되고, $\alpha$ 는 관성계수가 된다.

In [ ]:
class Momentum:

    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

# NAG(Nesteou Accelated Gradient)

Momentum은 현재위치에서 관성과 Gradient의 반대방향을 합한다.
NAG는 Momentume을 공격적으로 변환한다. 현재위치에서 관성과 관성방향으로 움직인 후의 Gradient의 반대방향을 합하게 된다.

점화식은

$V_{n} = \alpha V_{n-1} - \eta \bigtriangledown f(x_n + \alpha V_{n-1}) $

$V_{-1} = 0 $

$X_{n+1} = X_n + V_n$

하지만 이 NAG는 $X_n$이 아닌 다른점에서 Gradient를 구하기 때문에 오차역전파를 사용하는 신경망에서는 구현하기 적합하지 않다. 대신 Bengio의 근사적 접근을 사용할 수 있다.

Bengio의 근사적 접근은 $X_n$의 궤적 대신 $X_n'$의 궤적으로 Gradient Dsent를 진행한다. 그러면 파라미터를 수정하지 않아도 되고 최종적으로는 관성이 줄어들며 $X_n$ 과 $X_n'$이 서로 근하하게 되므로 신경망에서 사용하기 적합하게 되는 것 이다.

NAG에서 Bengio의 근사적방법을 적용하는 방법은 노트 참고.

In [ ]:
class Nesterov:

    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] *= self.momentum
            self.v[key] -= self.lr * grads[key]
            params[key] += self.mometum * self.momentum * self.v[key]
            params[key] -= (1 + self.momentum) * self.lr * grads[key]